In [ ]:
from hyperparameterStudy.image_dataset import ImageDataset
from tensorflow.keras.applications import VGG16
import tensorflow as tf
import random
import tensorflow.keras as k


file_list = ImageDataset.load_file_list("train")[-170:]
dataset = ImageDataset(data_list=file_list, validation_split=True, mil=False, rgb=True)
for elem in dataset.dataset_train.take(1):
    print(elem[1])
    print(elem[0].shape)
# TODO: Memory Problem sind die 3 Channel

# Taken From:
# https://tvst.arvojournals.org/article.aspx?articleid=2770240



2022-06-20 14:02:43.534775: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-20 14:02:43.534792: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-20 14:02:43.591792: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-06-20 14:02:43.592403: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-06-20 14:02:43.606298: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-20 14:02:43.606424: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-20 14:02:43.606706: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-20 14:02:43.606713: I tensorflow/core/profiler/lib/profiler_session.cc:125] Prof

In [ ]:
def create_vgg_model(input_shape=(1444, 1448, 3), train_from_layer=14):
    network_input = k.layers.Input(shape=input_shape)
    model = VGG16(weights='imagenet', input_tensor=network_input, include_top=False)

    feature_extraction_model = k.models.Model(inputs=model.input, outputs=model.layers[train_from_layer].output)
    feature_extraction_model.trainable = True
    classify_model = k.models.Model(inputs=model.layers[train_from_layer].output, outputs=model.output)

    pooling_layer = k.layers.GlobalAveragePooling2D()
    dropout_layer = k.layers.Dropout(0.1)
    dense_layer1 = k.layers.Dense(64, activation="relu")
    dense_layer2 = k.layers.Dense(1, activation="linear")
    model = k.Sequential([feature_extraction_model, classify_model, pooling_layer, dropout_layer, dense_layer1,
                          dense_layer2])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.summary()
    return model

In [3]:
import Callbacks
import importlib
import tensorflow.keras as k
importlib.reload(Callbacks)

vgg_model = create_vgg_model()
vgg_model.fit(dataset.dataset_train.batch(2), validation_data=dataset.dataset_val.batch(1), epochs=15, callbacks=Callbacks.vgg_callback("vgg2layers"))

2022-06-20 13:56:46.414198: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-20 13:56:46.414216: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-20 13:56:46.479401: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-06-20 13:56:46.479649: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-06-20 13:56:46.488521: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-20 13:56:46.488620: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-20 13:56:46.488875: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-20 13:56:46.488882: I tensorflow/core/profiler/lib/profiler_session.cc:125] Prof

NameError: name 'k' is not defined

In [1]:
from hyperparameterStudy.hyper_postprocessing import HyperPostprocessor
from hyperparameterStudy.image_dataset import ImageDataset

file_list = ImageDataset.load_file_list("test")
dataset = ImageDataset(data_list=file_list, validation_split=False, mil=False, rgb=True)
model_name = "vgg2layers"
hp = HyperPostprocessor(model_name, dataset, file_list, results_folder="results/vgg", history_folder="results")
hp.processing()

2022-06-15 19:10:46.505724: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-15 19:10:46.505740: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-15 19:10:46.567646: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-06-15 19:10:46.568251: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-06-15 19:10:46.582163: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-15 19:10:46.582545: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-15 19:10:46.583406: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-15 19:10:46.583446: I tensorflow/core/profiler/lib/profiler_session.cc:125] Prof

 5/28 [====>.........................] - ETA: 1s

2022-06-15 19:10:53.701125: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


28/28 [==============================] - 6s 60ms/step


In [3]:
import tensorflow.keras as k
model = k.models.load_model("results/vgg/models/vgg2layers")

In [7]:
model.layers[1].layers[3]